In [ ]:
import pandas as pd
import numpy as np
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [ ]:
champions = pd.read_csv('champion.csv', header=None)

In [ ]:
champion_dic = {x: y for x, y in zip(champions[0], champions[1])}

In [ ]:
age = {}

In [ ]:
req_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36",
    "Accept": "*/*"
}


# Per year

In [ ]:
def get_by_year(year):

    url = f"https://www.basketball-reference.com/playoffs/NBA_{year}_totals.html"

    # Send a GET request to the URL
    res = requests.get(url, headers=req_headers)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(res.content, "html.parser")

    # Find the table containing the team playoff totals
    table = soup.find("table", {"id": "totals_stats"})

    # Get the headers of the table
    headers = [th.get_text() for th in table.find("tr").find_all("th")]

    # Get the data rows of the table
    rows = []
    for tr in table.find_all("tr")[1:]:
        rows.append([td.get_text() for td in tr.find_all("td")])

    # Create a pandas dataframe from the data
    df = pd.DataFrame(rows, columns=headers[1:])

    # Display the resulting dataframe
#     print(df.head())

    sub_df = df[df.Tm == champion_dic[year]]

    ages = pd.to_numeric(sub_df.Age)
    mps = pd.to_numeric(sub_df.MP)

    age[year] = ages.mean(), sum(ages * mps) / sum(mps) if sum(mps) > 0 else -1
    print(year, age[year])
    time.sleep(5)

In [ ]:
for year in champion_dic:
    if year not in age:
        get_by_year(year)

In [ ]:
years = sorted(age)

In [ ]:
df = pd.DataFrame.from_dict({year: [champion_dic[year], *age[year]] for year in years}, orient='index', columns=['team', '平均年龄', '按出场时间加权平均年龄'])

In [ ]:
df['diff'] = df['平均年龄'] - df['按出场时间加权平均年龄']

In [ ]:
df.sort_values('按出场时间加权平均年龄', ascending=False).head(10)